# Hidden Markov Model para glosado Otomi

In [1]:
from nltk.tag import hmm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from pickle import load
from itertools import chain
import pandas as pd
import numpy as np

In [2]:
file1 = load(open('pickle_objects/predata','rb'))
print(file1[0], len(file1))

[[['n', 'v', 'B-psd'], ['d', 'v', 'B-1.cpl'], ['ó', 'v', 'I-1.cpl'], ['p', 'v', 'B-stem'], ['h', 'v', 'I-stem'], ['μ', 'v', 'I-stem'], ['d', 'v', 'I-stem'], ['i', 'v', 'I-stem']], [['d', 'v', 'B-1.cpl'], ['ó', 'v', 'I-1.cpl'], ['p', 'v', 'B-stem'], ['ε', 'v', 'I-stem'], ['p', 'v', 'I-stem'], ['h', 'v', 'I-stem'], ['í', 'v', 'I-stem']], [['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['t', 'v', 'B-lig'], ["'", 'v', 'B-stem'], ['μ', 'v', 'I-stem'], ['n', 'v', 'I-stem'], ['g', 'v', 'B-1.obj'], ['í', 'v', 'I-1.obj']], [['b', 'v', 'B-3.cpl'], ['i', 'v', 'I-3.cpl'], ['m', 'v', 'B-stem'], ['ä', 'v', 'I-stem'], ['h', 'v', 'I-stem'], ['t', 'v', 'I-stem'], ['r', 'v', 'I-stem'], ['a', 'v', 'I-stem'], ['t', 'v', 'I-stem'], ['á', 'v', 'I-stem'], ['g', 'v', 'B-1.obj'], ['í', 'v', 'I-1.obj']], [['k', 'obl', 'B-stem'], ['o', 'obl', 'I-stem']], [['c', 'obl', 'B-stem'], ['h', 'obl', 'I-stem'], ['í', 'obl', 'I-stem'], ['k', 'obl', 'I-stem'], ['ó', 'obl', 'I-stem'], ['h', 'obl', 'I-stem'], ['t', 'obl', 'I-

In [3]:
data = []

for s in file1:
    sent_data = []
    for w in s:
        #print(w)
        word = [(x[0], x[2]) for x in w] + [(' ', ' ')]
        sent_data.append(word)
    
    one_sent = list(chain(*sent_data))
    one_sent.pop(-1)
    data.append(list(zip(*one_sent)))

In [4]:
#Data split 33%
train_pairs, eval_pairs = train_test_split(data, test_size=0.33)
print(len(train_pairs), len(eval_pairs))

#Separar el train en dos conjuntos
train_set = [list(zip(s,t)) for s,t in train_pairs]

1196 590


### Entrenamiento del modelo

In [5]:
%%time
trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_set)

CPU times: user 48.8 ms, sys: 821 µs, total: 49.6 ms
Wall time: 48.8 ms


### Evaluación

In [6]:
y_pred = []
y_true = []
for sent in eval_pairs:
    prediccion = tagger.tag(sent[0])
    y_pred.append(prediccion)
    y_true.append(sent[1])

In [7]:
Y_true = list(chain(*y_true))
Y_pred = list(chain(*y_pred))
chars,Y_pred = zip(*Y_pred) 
Y_pred = list(Y_pred)

In [8]:
labels = list(set(Y_true))
labels.remove(' ')

print('Accuracy:', accuracy_score(Y_true, Y_pred))

Accuracy: 0.8665164037064863


In [9]:
prec, rec, f1, supp = precision_recall_fscore_support(Y_true, Y_pred, labels=labels)

/home/mijangos/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
results = pd.DataFrame(data=np.array([prec, rec, f1, supp]).T, index=labels, columns=['Precision','Recall','F1', 'Support'])
results

,Precision,Recall,F1,Support
B-3.pss,0.000000,0.000000,0.000000,15.0
I-lig,0.000000,0.000000,0.000000,1.0
I-1.irr,0.000000,0.000000,0.000000,4.0
B-agujerear/v,1.000000,0.500000,0.666667,2.0
B-encl,0.000000,0.000000,0.000000,1.0
...,...,...,...,...
B-3.pss.pl,0.000000,0.000000,0.000000,1.0
I-stem,0.936846,0.833383,0.882091,6764.0
I-dual,0.777778,0.875000,0.823529,8.0
I-mod,0.178571,0.483871,0.260870,31.0


In [11]:
print(results.to_string())

                  Precision    Recall        F1  Support
B-3.pss            0.000000  0.000000  0.000000     15.0
I-lig              0.000000  0.000000  0.000000      1.0
I-1.irr            0.000000  0.000000  0.000000      4.0
B-agujerear/v      1.000000  0.500000  0.666667      2.0
B-encl             0.000000  0.000000  0.000000      1.0
I-muy              0.631579  0.800000  0.705882     30.0
I-gen              0.272727  1.000000  0.428571      6.0
B-mod              0.240000  0.300000  0.266667     20.0
I-pl.exc           0.721649  0.958904  0.823529     73.0
B-3.pot            0.974359  0.938272  0.955975     81.0
B-lim              0.888000  0.948718  0.917355    117.0
B-pues             1.000000  0.428571  0.600000      7.0
B-1.cpl.irr        1.000000  0.750000  0.857143      4.0
B-psd              0.945946  0.972222  0.958904    144.0
B-2.pot            1.000000  1.000000  1.000000      3.0
I-comp             0.000000  0.000000  0.000000      1.0
I-1.pot            0.835443  0.

In [12]:
aver_prec = (supp*prec).sum()/supp.sum()
aver_rec = (supp*rec).sum()/supp.sum()
aver_f1 = (supp*f1).sum()/supp.sum()

print('Average precision: {}\nAverage recall: {}\nAverage F1: {}'.format(aver_prec, aver_rec, aver_f1))

Average precision: 0.8790951172003872
Average recall: 0.845562087234197
Average F1: 0.8545884747700223
